# TP2

In [1]:
import re
import tkinter as tk
import selenium
import requests

from IPython.display import clear_output
from selenium import webdriver
from bs4 import BeautifulSoup

In [13]:
# Déclarations des fonctions
def rchop(thestring, ending):
    if thestring.endswith(ending):
        return thestring[:-len(ending)]
    return thestring

def info(*args, **kwargs):
    rulesArg = kwargs.get("rulesArg", False)
    wordArg = kwargs.get("wordArg", True)
    resultArg = kwargs.get("resultArg", True)
    
    if(rulesArg):
        print(f'\nLes règles :')
        pprint(rules)

    if(wordArg):
        print(f'\nLe mot : {testWord}')

    if(resultArg):
        print(f'\nLes dérivations :')
        for n in result.items():
            print(str(n).replace(',', ' ->'))

def check_words(bla, tss, *args, **kwargs):
    verbose = kwargs.get("verbose", False)
    if(bla): # if jdm-warning exist (not a word)
        if(verbose):
            print(f'le mot {tss} n\'existe pas\n')
    else: # else (word exist)
        if(verbose):
            print(f'le mot {tss} existe\n')
        return tss

In [3]:
# Déclarations des variables & lecture mot & nature du mot
nature = {'Verbe': 'VERBE', 'Nom': 'NOM', 'Adjectif': 'ADJ', 'Nom propre': 'NOMPROPRE'}
rules = []
result = {}

# Reading the word
print(f'Enter your word : ')
testWord = input()

# Clear output
clear_output(wait = False)

# Reading the word's nature
print(f'Chose your word\'s nature : ')
#pprint(nature)
for(key, value) in nature.items():
    print(f'\t{value} for {key}')
natureWord = input()

# Clear output
clear_output(wait = False)

# Ouverture du fichier
file = open("rules.txt", "r")

# Lecture du fichier & enregistrement des règles
if file.mode == 'r':
    lines = file.readlines()
    for line in lines:
        rules.append(line.rstrip("\n").split("==>"))

# Fermeture du fichier
file.close()

# Génération des mots et leur nature
for rule in rules:
    ruleG = rule[0].split(":")
    ruleD = rule[1].split(":")
    match = re.match(".*" + ruleG[1], testWord)
    if match and ruleG[0] == natureWord:
        if ruleG[1] != "":
            generatedWord = rchop(testWord, ruleG[1]) + ruleD[1]
            result[generatedWord] = ruleD[0]
        else:
            generatedWord = testWord + ruleD[1]
            result[generatedWord] = ruleD[0]


# Display : set arguments (rulesArg, wordArg and resultArg) to false to desactivate displaying corresponding information
info(rulesArg = False, wordArg = False, resultArg = False)

In [16]:
final = []
for word in result:
    # Send a request to jdm
    url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(word) + "&rel="
    
    # Get the response
    response = requests.get(url)
    
    # bs4
    soup = BeautifulSoup(response.text)
    
    # Searching for jdm-warning
    mydivs = soup.findAll("div", {"class": "jdm-warning"})
    
    final.append(check_words(mydivs, word, verbose = False))
    final = [i for i in final if i] 
    
if(final):
    print('Les mots valables sont :')
    for f in final:
        print(f'\t{f} : {result[f]}')
else:
    print('Aucun mot n\'est valable')

Les mots valables sont :
	Chantage : NOM
	Chanteur : AGENT
	Chanteuse : AGENT
	Chante : NOM


In [ ]:
#Envoie de requete à jeux de mots
url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(testWord) + "&rel="

# Using Chrome to access web
driver = webdriver.Chrome(executable_path=r'/mnt/c/Program Files/chromedriver/chromedriver.exe')

# Open the website
driver.get(url)

# Find download button
jdmSection = driver.find_element_by_class_name('jdm-warning')
print(jdmSection)